<h1>Chapter 06. Object references, modifiability and reuse.</h1>

<h2>Variables are not boxes</h2>

Variables `a` and `b` store references to the same list, not copies of the list

In [1]:
a = [1, 2, 3]
b = a

b.append(4)
b

[1, 2, 3, 4]

Variables are bounded to objects only after objects are created

In [2]:
class Gizmo:
    def __init__(self):
        print(f"Gizmo id: {id(self)}")

In [3]:
x = Gizmo()

Gizmo id: 4454899856


In [4]:
try:
    y = Gizmo() * 10
except TypeError as e:
    print(e.__repr__())

Gizmo id: 4454814576
TypeError("unsupported operand type(s) for *: 'Gizmo' and 'int'")


The second `Gizmo` object was still created before the attempt to perform multiplication. But the `y` varaible was never created because an exeception occured when the right part was being calculated.

<h2>Identity, Equality and Pseudonyms</h2>

In [5]:
charles = {
    'name': 'Charles L. Dodgson',
    'born': 1832
}

In [6]:
lewis = charles

lewis is charles

True

In [7]:
id(charles), id(lewis)

(4454991936, 4454991936)

In [8]:
lewis['balance'] = 950
charles

{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

In [9]:
alex = {
    'name': 'Charles L. Dodgson',
    'born': 1832,
    'balance': 950
}

alex == charles

True

In [10]:
alex is charles

False

<h3>Choice between <code>==</code> and <code>is</code></h3>

The `==` operator compares the values of objects (the data stored in them), and the `is` operator compares their identifiers.

<h3>Relative immutability of tuples</h3>

In [11]:
t1 = (1, 2, [3, 4])
t2 = (1, 2, [3, 4])

t1 == t2

True

In [12]:
id(t1[-1])

4454996672

In [13]:
t1[-1].append(5)
t1

(1, 2, [3, 4, 5])

In [14]:
id(t1[-1])

4454996672

In [15]:
t1 == t2

False

The object identifier `t1[-1]` has not changed only its value has changed.
`t1` and `t2` are now not equal.

<h2>By default, copying is superficial</h2>

In [16]:
l1 = [3, [55, 44], (7, 8, 9)]

In [17]:
l2 = list(l1)  # l2 = l1[:]

In [18]:
l2

[3, [55, 44], (7, 8, 9)]

In [19]:
l2 == l1

True

In [20]:
l2 is l1

False

<h3>Deep and superficial copying of objects</h3>

In [21]:
class Bus:
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

`copy.copy()` creates a shallow copy of the object, duplicating the object itself but not recursively copying inner objects.
`copy.deepcopy()` creates a deep copy of the object, recursively copying all inner objects as well, ensuring changes to the copied object won't affect the original.

In [22]:
import copy


bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)

In [23]:
id(bus1), id(bus2), id(bus3)

(4454901008, 4455042368, 4455049472)

In [24]:
id(bus1.passengers), id(bus2.passengers), id(bus3.passengers)

(4455143552, 4455143552, 4454549952)

In [25]:
bus1.drop('Bill')

In [26]:
bus2.passengers

['Alice', 'Claire', 'David']

In [27]:
bus3.passengers

['Alice', 'Bill', 'Claire', 'David']

In [28]:
bus2.pick('Julie')

In [29]:
bus1.passengers

['Alice', 'Claire', 'David', 'Julie']

In [30]:
bus3.passengers

['Alice', 'Bill', 'Claire', 'David']

Cyclic references: `b` refers to `a` and then appends to the end of `a`; however `deepcopy` handles the copying of `a`

In [31]:
a = [10, 20]
b = [a, 30]

In [32]:
a.append(b)
a

[10, 20, [[...], 30]]

In [33]:
c = copy.deepcopy(a)
c

[10, 20, [[...], 30]]

<h2>Function parameters as references</h2>

A function can modify any modifiable object passed to it

In [34]:
def f(a, b):
    a += b
    return a

In [35]:
x = 1
y = 2

f(x, y)

3

In [36]:
x, y  # x has not changed

(1, 2)

In [37]:
a = [1, 2]
b = [3, 4]

f(a, b)

[1, 2, 3, 4]

In [38]:
a, b  # list a has changed

([1, 2, 3, 4], [3, 4])

In [39]:
c = (10, 20)
d = (30, 40)

f(c, d)

(10, 20, 30, 40)

In [40]:
c, d  # tuple c has not changed

((10, 20), (30, 40))

<h3>Default values of the modifiable type: bad idea</h3>

A simple class illustrating the dangers of modifiable default values

In [41]:
class HauntedBus:
    def __init__(self, passengers=[]):
        self.passengers = passengers

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [42]:
bus1 = HauntedBus(['Alice', 'Bill'])
bus1.passengers

['Alice', 'Bill']

In [43]:
bus1.pick('Charlie')
bus1.drop('Alice')
bus1.passengers

['Bill', 'Charlie']

`bus2` is initially empty, so the `self.passengers` attribute is assigned an empty list by default

In [44]:
bus2 = HauntedBus()
bus2.pick('Carrie')
bus2.passengers

['Carrie']

The list is already not empty by default 

In [45]:
bus3 = HauntedBus()
bus3.passengers

['Carrie']

`Dave` from `bus3` is also in `bus2`

In [46]:
bus3.pick('Dave')
bus2.passengers

['Carrie', 'Dave']

In [47]:
bus2.passengers is bus3.passengers

True

In [48]:
bus1.passengers

['Bill', 'Charlie']

The problem is that all `HauntedBus` instances whose constructor has not been explicitly passed a passenger list share the same default list.

<h3>Safety programming in the presence of changeable parameters</h3>

In [49]:
class TwilightBus:
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            # self.passengers becomes a pseudonym of the passengers parameter,
            # wich itself is a pseudonym of the actual argument passed to
            # the __init__ method
            self.passengers = passengers

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [50]:
bascketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']

In [51]:
bus = TwilightBus(bascketball_team)
bus.drop('Tina')
bus.drop('Pat')

bascketball_team

['Sue', 'Maya', 'Diana']

The problem is that the `bus` object creates a pseudonym of the list passed to the constructor. But it should store its own list of passengers. To fix the error: in the `__init__` method create the copy of the passengers list or convert it to the list type, if the parameter has a different type, as was done in the example `class Bus`. 

<h2><code>del</code> and the collection of trash</h2>

`del` used to remove a reference to an object.

In [52]:
a = [1, 2]
b = a

del a

In [53]:
b

[1, 2]

In [54]:
b = [3]
b

[3]

In [55]:
try:
    print(a)
except NameError as e:
    print(e.__repr__())

NameError("name 'a' is not defined")


To demonstrate the end of the objext's life, next example uses the `weakref.finalize` function, which register a callback function that is called before the object is destroyed.

In [56]:
from weakref import finalize


s1 = {1, 2, 3}
s2 = s1

In [57]:
def bye():
    print('The object has been deleted.')

In [58]:
ender = finalize(s1, bye)
ender.alive

True

In [59]:
del s1  # del does not delete an object, but only a reference to it

In [60]:
ender.alive

True

In [61]:
s2 = 'spam'

The element was deleted...


In [62]:
ender.alive

False